In [1]:
import pandas as pd
import numpy as np
import json
import os
from glob import glob
import cv2
import sys

## Json파일 불러오기

In [2]:
with open('result_md_v4.1.0.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [3]:
print(json_data) # 출력이 너무 많아서 못띄움

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from pprint import pprint # 차례대로 출력
pprint(json_data) # 정지로 끌것.

In [5]:
json_data['images'][4] # 바운딩 박스가 그려지지 않은 사진의 정보

{'file': '/dataset/knps_03/images_and_annotations/all/설악_생태통로_080_너구리_20191010_0059 09.jpg',
 'max_detection_conf': 0,
 'detections': []}

In [6]:
json_data['images'][0] # 바운딩 박스가 한 개 그려진 사진의 정보

{'file': '/dataset/knps_03/images_and_annotations/all/소북_생태통로_001_멧돼지_20191107_2303 새끼 02.jpg',
 'max_detection_conf': 0.919,
 'detections': [{'category': '1',
   'conf': 0.919,
   'bbox': [0.5313, 0.5207, 0.02388, 0.07787]}]}

In [10]:
json_data['images'][22] # 바운딩 박스가 두 개 그려진 사진의 정보

{'file': '/dataset/knps_03/images_and_annotations/all/멧돼지 20190801 2105 6(새끼5) 13.jpg',
 'max_detection_conf': 0.983,
 'detections': [{'category': '1',
   'conf': 0.983,
   'bbox': [0.3125, 0.5581, 0.0824, 0.1299]},
  {'category': '1', 'conf': 0.889, 'bbox': [0.535, 0.4652, 0.0634, 0.1768]}]}

In [13]:
print(json_data['images'][22]['file'][44:][:-4]) # 파일명 추출
print(json_data['images'][22]['max_detection_conf']) # 바운딩 박스가 여러 개일 경우의 conf 최댓값
print(json_data['images'][22]['detections']) # 바운딩 박스의 정보
print(json_data['images'][22]['detections'][0]) # 첫 번째 바운딩 박스의 정보
print(json_data['images'][22]['detections'][0]['bbox']) #첫 번째 바운딩 박스의 좌표
print(json_data['images'][22]['detections'][1]) # 두 번째 바운딩 박스의 정보
print(json_data['images'][22]['detections'][1]['bbox']) #두 번째 바운딩 박스의 좌표

멧돼지 20190801 2105 6(새끼5) 13
0.983
[{'category': '1', 'conf': 0.983, 'bbox': [0.3125, 0.5581, 0.0824, 0.1299]}, {'category': '1', 'conf': 0.889, 'bbox': [0.535, 0.4652, 0.0634, 0.1768]}]
{'category': '1', 'conf': 0.983, 'bbox': [0.3125, 0.5581, 0.0824, 0.1299]}
[0.3125, 0.5581, 0.0824, 0.1299]
{'category': '1', 'conf': 0.889, 'bbox': [0.535, 0.4652, 0.0634, 0.1768]}
[0.535, 0.4652, 0.0634, 0.1768]


In [15]:
# 바운딩 박스 좌표 욜로 포맷으로 출력 연습
bbox='0'
for i in range(4):
    bbox+=' '+str(json_data['images'][0]['detections'][0]['bbox'][i])
print(bbox)

0 0.5313 0.5207 0.02388 0.07787


In [3]:
# 데이터프레임 생성
df=pd.DataFrame(columns=(['file_name', 'max_conf', 'category', 'conf', 'bbox']))
for i in range(len(json_data['images'])): # 사진 갯수만큼 반복
    file_name=json_data['images'][i]['file'][44:][:-4]
    max_conf=json_data['images'][i]['max_detection_conf']
    category=[]
    conf=[]
    bbox=[]
    if max_conf != 0: # bbox값이 존재하면
        for j in range(len(json_data['images'][i]['detections'])): # bbox가 여러개일 수 있으므로 반복문 # 2:22, 3:60, 4:570 5:113 6:3005
            category.append(json_data['images'][i]['detections'][j]['category'])
            conf.append(json_data['images'][i]['detections'][j]['conf'])
            bbox.append(json_data['images'][i]['detections'][j]['bbox'])
    else: # bbox값이 존재하지 않으면 공백처리
        category=np.nan
        conf=np.nan
        bbox=np.nan
    li=[file_name, max_conf, category, conf, bbox]
    df.loc[i]=li

C:\anaconda\lib\site-packages\pandas\core\dtypes\cast.py:881: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


In [18]:
print(df.shape)
pprint(df.info())
df.head()

(60234, 5)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 60234 entries, 0 to 60233
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   file_name  60234 non-null  object 
 1   max_conf   60234 non-null  float64
 2   category   30213 non-null  object 
 3   conf       30213 non-null  object 
 4   bbox       30213 non-null  object 
dtypes: float64(1), object(4)
memory usage: 2.8+ MB
None


,file_name,max_conf,category,conf,bbox
0,소북_생태통로_001_멧돼지_20191107_2303 새끼 02,0.919,[1],[0.919],"[[0.5313, 0.5207, 0.02388, 0.07787]]"
1,소북_생태통로_001_고라니_20190808_1521 08,0.981,[1],[0.981],"[[0.02443, 0.4087, 0.142, 0.2411]]"
2,소북_생태통로_001_너구리_20190806_2004 (2) 07,0.998,[1],[0.998],"[[0.5401, 0.3977, 0.08631, 0.228]]"
3,멧돼지 20190627 1540 7(새끼6) 13,0.938,[1],[0.938],"[[0.5857, 0.4113, 0.07176, 0.05283]]"
4,설악_생태통로_080_너구리_20191010_0059 09,0.000,NaN,NaN,NaN


In [19]:
df=df.sort_values('file_name')
df

,file_name,max_conf,category,conf,bbox
31449,0322 너구리 01,0.999,[1],[0.999],"[[0.3983, 0.738, 0.2246, 0.196]]"
32791,0322 너구리 02,0.999,[1],[0.999],"[[0.398, 0.7384, 0.2246, 0.1944]]"
55365,0322 너구리 03,0.999,[1],[0.999],"[[0.4189, 0.7133, 0.2404, 0.2192]]"
1359,0322 너구리 04,0.999,[1],[0.999],"[[0.4373, 0.7272, 0.165, 0.2061]]"
8133,0322 너구리 05,0.999,[1],[0.999],"[[0.4975, 0.7131, 0.09413, 0.1703]]"
...,...,...,...,...,...
7786,하늘다람쥐170103 0628 39,0.000,NaN,NaN,NaN
7043,하늘다람쥐170103 0628 40,0.000,NaN,NaN,NaN
12837,하늘다람쥐170103 0628 41,0.000,NaN,NaN,NaN
21866,흰배지빠귀 20190501 0818,0.999,[1],[0.999],"[[0.4533, 0.5306, 0.2459, 0.3038]]"


###### 데이터프레임을 csv파일로 저장

In [20]:
# df.to_csv('json.csv', header=True, index=False) # 칼럼명은 남기고 인덱스는 버림

## 라벨링

In [49]:
# 멧돼지 데이터만 추출
wildboar=df[df.file_name.str.contains('멧돼지')]
print(f'받은 사진 데이터 총 개수 : {len(wildboar)}개')
print(f'받은 멧돼지 사진 데이터 개수 : {len(wildboar)}개')
print(f'받은 멧돼지 사진 데이터 중 멧돼지 라벨링이 되어있는 데이터 개수 : {len(wildboar.dropna())}개')

받은 사진 데이터 총 개수 : 28191개
받은 멧돼지 사진 데이터 개수 : 28191개
받은 멧돼지 사진 데이터 중 멧돼지 라벨링이 되어있는 데이터 개수 : 12665개


In [50]:
# 고라니 데이터만 추출
gorani=df[df.file_name.str.contains('고라니')]
print(f'받은 사진 데이터 총 개수 : {len(gorani)}개')
print(f'받은 멧돼지 사진 데이터 개수 : {len(gorani)}개')
print(f'받은 멧돼지 사진 데이터 중 멧돼지 라벨링이 되어있는 데이터 개수 : {len(gorani.dropna())}개')

받은 사진 데이터 총 개수 : 17339개
받은 멧돼지 사진 데이터 개수 : 17339개
받은 멧돼지 사진 데이터 중 멧돼지 라벨링이 되어있는 데이터 개수 : 11759개


In [51]:
# na 제거
wildboar=wildboar.dropna().reset_index().drop('index', axis=1)
gorani=gorani.dropna().reset_index().drop('index', axis=1)

In [55]:
# sample
for i in range(10): #10개만
    if wildboar.bbox[i]!=wildboar.bbox[i]: # wildboar.bbox가 존재하지 않으면
        pass
    else: #wildboar.bbox가 존재하면
        for j in range(len(wildboar.bbox[i])): # 박스 개수만큼
            s='0' # 라벨링 카테고리
            for k in range(len(wildboar.bbox[i][j])): # bbox 좌표 4개 반복문
                s+=' '
                s+=str(wildboar.bbox[i][j][k])
            if j==0: # 박스가 하나면
                st=s
            else: # 박스가 둘 이상이면
                st+='\n'
                st+=s
        print(st)

0 0.5198 0.4216 0.1715 0.1715
0 0.5058 0.3662 0.06482 0.2542
0 0.3491 0.4037 0.182 0.2525
0 0.3425 0.4159 0.1941 0.2267
0 0.4292 0.4053 0.1741 0.2186
0 0.4823 0.3853 0.214 0.2242
0 0.6716 0.3586 0.1965 0.2249
0 0.6087 0.418 0.1632 0.17
0 0.862 0.3405 0.1354 0.2098
0 0.4712 0.4204 0.1463 0.1851


In [56]:
# bbox 두 개인 경우
i=15
for j in range(len(wildboar.bbox[i])): # 박스 개수만큼
    s='0' # 라벨링 카테고리
    for k in range(len(wildboar.bbox[i][j])): # bbox 좌표 4개 반복문
        s+=' '
        s+=str(wildboar.bbox[i][j][k])
    if j==0: # 박스가 하나면
        st=s
    else: # 박스가 둘 이상이면
        st+='\n'
        st+=s
print(st)

0 0.549 0.4421 0.04337 0.1319
0 0.5068 0.4416 0.08403 0.14


In [59]:
os.mkdir('test_test')

In [ ]:
# # 라벨링 최종 ver멧돼지
# os.makedirs('label_wildboar', exist_ok=True) # 폴더 생성
# for i in range(len(wildboar)):
#     if wildboar.bbox[i]!=wildboar.bbox[i]: # wildboar.bbox가 존재하지 않으면
#         pass
#     else: #wildboar.bbox가 존재하면
#         for j in range(len(wildboar.bbox[i])): # 박스 개수만큼
#             s='0' # 라벨링 카테고리 멧돼지:0, 고라니:1
#             for k in range(len(wildboar.bbox[i][j])): # bbox 좌표 4개 반복문
#                 s+=' '
#                 s+=str(wildboar.bbox[i][j][k])
#             if j==0: # 박스가 하나면
#                 st=s
#             else: # 박스가 둘 이상이면
#                 st+='\n'
#                 st+=s
#             with open('label_wildboar/'+sample.file_name[i]+'.txt', 'w') as f: # txt파일로 저장
#                 f.write(st)

In [ ]:
# # ver 고라니
# os.makedirs('label_gorani', exist_ok=True) # 폴더 생성
# for i in range(len(gorani)):
#     if gorani.bbox[i]!=gorani.bbox[i]: # gorani.bbox가 존재하지 않으면
#         pass
#     else: # gorani.bbox가 존재하면
#         for j in range(len(gorani.bbox[i])): # 박스 개수만큼
#             s=' 1' # 라벨링 카테고리 멧돼지:0, 고라니:1
#             for k in range(len(gorani.bbox[i][j])): # bbox 좌표 4개 반복문
#                 s+=' '
#                 s+=str(gorani.bbox[i][j][k])
#             if j==0: # 박스가 하나면
#                 st=s
#             else: # 박스가 둘 이상이면
#                 st+='\n'
#                 st+=s
#             with open('label_gorani/'+sample.file_name[i]+'.txt', 'w') as f: # txt파일로 저장
#                 f.write(st)

## 라벨링이 되어있는 사진만 남기기
+ image_wildboar 폴더와
+ image_gorani 폴더가 존재하는 경우 시행

In [70]:
jpg_list_wildboar=glob('image_wildboar/*.jpg') # 멧돼지 사진 목록 불러오기
for i in range(len(jpg_list_wildboar)):
    jpg_list_wildboar[i]=jpg_list_wildboar[i].replace('\\','/') # 역슬래시 슬래시로 통일
print(len(jpg_list_wildboar))
print(type(jpg_list_wildboar))
print(jpg_list_wildboar[0:2])
# 실제로는 28191개

9197
<class 'list'>
['image_wildboar/1-멧돼지 2마리-하 01.jpg', 'image_wildboar/1-멧돼지 2마리-하 02.jpg']


In [73]:
jpg_list_wildboar[4522][15:-4] # 샘플 확인

'소북_생태통로_001_멧돼지_20190925_0622 (1) 03'

In [74]:
for i in range(len(jpg_list_wildboar)): 
    jpg_list_wildboar[i]=jpg_list_wildboar[i][15:-4] # 경로&확장자 제거
jpg_list_wildboar[3385]

'소북_생태통로_001_멧돼지_20190802_1942-51 5(새끼4) (2) 05'

In [80]:
txt_list_wildboar=glob('label_wildboar/*.txt') # 멧돼지 라벨 목록 불러오기
for i in range(len(txt_list_wildboar)):
    txt_list_wildboar[i]=txt_list_wildboar[i].replace('\\','/') # 역슬래시 슬래시로 통일
print(len(txt_list_wildboar))
print(type(txt_list_wildboar))
print(txt_list_wildboar[0:2])

12665
<class 'list'>
['label_wildboar/1-경계하며 이동하는 멧돼지-상 07.txt', 'label_wildboar/1-달리는멧돼지-하 18.txt']


In [81]:
txt_list_wildboar[42][15:-4] # 샘플 확인

'1-멧돼지-하 (6) 06'

In [82]:
for i in range(len(txt_list_wildboar)):
    txt_list_wildboar[i]=txt_list_wildboar[i][15:-4] # 경로&확장자 제거
txt_list_wildboar[335]

'5-멧돼지-1 04'

In [ ]:
# for i in range(len(jpg_list_wildboar)):
#     if jpg_list_wildboar[i] not in txt_list_wildboar: # 멧돼지 사진 파일이름이 라벨링되어있지 않으면
#         os.remove(jpg_list_wildboar[i]) # 사진 파일 삭제

In [89]:
jpg_list_gorani=glob('image_gorani/*.jpg') # 고라니 사진 목록 불러오기
for i in range(len(jpg_list_gorani)):
    jpg_list_gorani[i]=jpg_list_gorani[i].replace('\\','/') # 역슬래시 슬래시로 통일
print(len(jpg_list_gorani))
print(type(jpg_list_gorani))
print(jpg_list_gorani[0:2])
# 실제로는 17339개
txt_list_gorani=glob('label_gorani/*.txt') # 고라니 라벨 목록 불러오기
for i in range(len(txt_list_gorani)):
    txt_list_gorani[i]=txt_list_gorani[i].replace('\\','/') # 역슬래시 슬래시로 통일
print(len(txt_list_gorani))
print(type(txt_list_gorani))
print(txt_list_gorani[0:2])

10526
<class 'list'>
['image_gorani/1-고라니 (2) 01.jpg', 'image_gorani/1-고라니 (2) 02.jpg']
11759
<class 'list'>
['label_gorani/1-고라니 (2) 01.txt', 'label_gorani/1-고라니 (2) 02.txt']


In [91]:
# 샘플 확인
print(jpg_list_gorani[78][13:-4])
print(txt_list_gorani[500][13:-4])

1-주변을 경계하는 고라니-상 12
5-고라니 (2) 08


In [92]:
# 경로&확장자 제거
for i in range(len(jpg_list_gorani)): 
    jpg_list_gorani[i]=jpg_list_gorani[i][13:-4]
print(jpg_list_gorani[775])
for i in range(len(txt_list_gorani)):
    txt_list_gorani[i]=txt_list_gorani[i][13:-4]
print(txt_list_gorani[395])

고라니 190408 0500-03 (1) 09
3-고라니-하 (2) 04


In [ ]:
# for i in range(len(jpg_list_gorani)):
#     if jpg_list_gorani[i] not in txt_list_gorani: # 멧돼지 사진 파일이름이 라벨링되어있지 않으면
#         os.remove(jpg_list_gorani[i]) # 사진 파일 삭제

## 사진과 bbox 띄우기

#### 포맷 확인

In [103]:
#practice MegaDetector
i=4522
img_array = np.fromfile('image_wildboar/'+jpg_list_wildboar[i]+'.jpg', np.uint8) # 한글제목을 불러오기 위한 디코딩
img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
fy, fx, _ = img.shape # 사진 크기값 받아오기
x,y,w,h = 0.4747, 0.2639, 0.4307, 0.4169
x1=round(x*fx)
x2=round((x+w)*fx)
y1=round(y*fy)
y2=round((y+h)*fy)
cv2.rectangle(img, (x1,y1), (x2,y2), (0,0,255), 5) # 네모 사진 그리기, 사진(img)에 좌표 두개를 이용한 사각형, bgr칼라, 두께
cv2.imshow(str(i), img)
cv2.waitKey()
cv2.destroyAllWindows()

In [97]:
#practice YOLOv5
i=4522 
img_array = np.fromfile('image_wildboar/'+jpg_list_wildboar[i]+'.jpg', np.uint8)
img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
fy, fx, _ = img.shape
x,y,w,h = 0.4747, 0.2639, 0.4307, 0.4169
x1=round((x-0.5*w)*fx)
x2=round((x+0.5*w)*fx)
y1=round((y-0.5*h)*fy)
y2=round((y+0.5*h)*fy)
cv2.rectangle(img, (x1,y1), (x2,y2), (0,0,255),5)
cv2.imshow(str(i), img)
cv2.waitKey()
cv2.destroyAllWindows()

In [98]:
#practice YOLOv5 좌표얻기
i=4522 
img_array = np.fromfile('image_wildboar/'+jpg_list_wildboar[i]+'.jpg', np.uint8)
img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
fy, fx, _ = img.shape
x,y,w,h = 0.4747, 0.2639, 0.4307, 0.4169
x1=round(x*fx)
x2=round((x+w)*fx)
y1=round(y*fy)
y2=round((y+h)*fy)
x=(x2+x1)/(2*fx)
y=(y2+y1)/(2*fy)
w=(x2-x1)/fx
h=(y2-y1)/fy
print(x,y,w,h)

0.690234375 0.4722222222222222 0.43046875 0.4166666666666667


In [104]:
#practice YOLOv5
i=4522 
img_array = np.fromfile('image_wildboar/'+jpg_list_wildboar[i]+'.jpg', np.uint8) 
img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
fy, fx, _ = img.shape
x,y,w,h = 0.6902, 0.4722, 0.4305, 0.4167
x1=round((x-0.5*w)*fx)
x2=round((x+0.5*w)*fx)
y1=round((y-0.5*h)*fy)
y2=round((y+0.5*h)*fy)
cv2.rectangle(img, (x1,y1), (x2,y2), (0,0,255),5)
cv2.imshow(str(i), img)
cv2.waitKey()
cv2.destroyAllWindows()

#### 좌표값을 텍스트 파일에서 직접 불러오기

In [106]:
#practice ver
i=4522
img_array = np.fromfile('image_wildboar/'+jpg_list_wildboar[i]+'.jpg', np.uint8) # 한글제목을 불러오기 위한 디코딩
img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
fy, fx, _ = img.shape # 사진 크기 불러오기, 칼라여부는 안받아옴
with open('label_wildboar/'+jpg_list_wildboar[i]+'.txt', 'r') as f: # 텍스트파일 불러오기
    code=f.read().split('\n')
for j in range(len(code)):
    _,x,y,w,h=code[j].split(' ') # 처음은 카테고리라 안받아옴
    x=float(x); y=float(y); w=float(w); h=float(h)
    x1=round(-fx/2*w+fx*x)
    x2=round(fx/2*w+fx*x)
    y1=round(-fy/2*h+fy*y)
    y2=round(fy/2*h+fy*y)
    cv2.rectangle(img, (x1,y1), (x2,y2), (0,0,255),5)
cv2.imshow(str(i), img)
cv2.waitKey()
cv2.destroyAllWindows()

In [113]:
# pro ver
# p를 누르면 다음사진 | press 'p' means show next sccne
# b를 누르면 이전사진 | press 'b' means show before scene
# r을 누르면 이상한 사진목록 세이브 | press 'r' means record strange scene name
# q를 누르면 종료 | press 'q' means quit system
start=4522
strange_list=[]
i=start
while i<len(jpg_list_wildboar): # 사진갯수만큼 프로그램 돌림
    img_array = np.fromfile('image_wildboar/'+jpg_list_wildboar[i]+'.jpg', np.uint8) #한글명 디코딩
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    fy, fx, _ = img.shape # 사진크기 받아옴, 칼라여부 안받음
    with open('label_wildboar/'+jpg_list_wildboar[i]+'.txt', 'r') as f: # 라벨링 txt파일 불러옴
        code=f.read().split('\n')
    for j in range(len(code)): # bbox가 두개 이상일 경우를 위해 반복문
        _,x,y,w,h=code[j].split(' ') # 카테고리 안받음
        x=float(x); y=float(y); w=float(w); h=float(h)
        x1=round(-fx/2*w+fx*x)
        x2=round(fx/2*w+fx*x)
        y1=round(-fy/2*h+fy*y)
        y2=round(fy/2*h+fy*y) # CV2 포맷
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,0,255), 5)
    cv2.imshow(str(i), img) # 사진띄우기; 제목, 사진
    cv2.moveWindow(str(i),-16,-31) # 창 위치 조절
    key=cv2.waitKey()
    if key == ord('q'): # q 누르면 종료
        break
    if key == ord('r'): # r 누르면 저장하고 다음 사진
        strange_list.append(jpg_list_wildboar[i]+'.jpg')
        i+=1
        cv2.destroyAllWindows()
    if key == ord('p'): # p 누르면 다음 사진
        i+=1
        cv2.destroyAllWindows()
    if key == ord('b'): # b 누르면 이전 사진
        i-=1
        cv2.destroyAllWindows()
    cv2.destroyAllWindows()
cv2.destroyAllWindows()
print(jpg_list_wildboar[i],'.jpg 까지 확인하였습니다.', sep='')
print(jpg_list_wildboar[i], f'.jpg 는 {i}번째이므로 다음에 돌리면 start값을 {i+1}로 주세요', sep='')

소북_생태통로_001_멧돼지_20190925_0622 (1) 08.jpg 까지 확인하였습니다.
소북_생태통로_001_멧돼지_20190925_0622 (1) 08.jpg 는 4527번째이므로 다음에 돌리면 start값을 4528로 주세요


In [114]:
# 저장된 이상한 사진 분리
s=''
for i in range(len(strange_list)):
    s+=strange_list[i]
    s+='\n'
# strange_wildboar.txt로 목록 저장
with open('strange_wildboar.txt', 'a') as f:
    f.write(s)

In [ ]:
# strange_wildboar.txt의 파일명 다 불러와서 file_list로 저장
with open('strange_wildboar.txt', 'r') as f:
    file_list=f.read().split('\n')

In [119]:
os.chdir('image_wildboar/')
os.getcwd()

'C:\\Users\\tripleler\\python\\NationalPark_Project\\image_wildboar'

In [115]:
# # 해당 사진 제거
# # 제거 전 디렉토리 변결할것!!
# for i in range(len(file_list)):
#     os.remove(file_list[i])

'소북_생태통로_001_멧돼지_20190925_0622 (1) 06.jpg\n소북_생태통로_001_멧돼지_20190925_0622 (1) 07.jpg\n'